In [1]:
import os
import torch
import torch.nn as nn
from tqdm import tqdm
import config as cfg
import load_s3dis_preorder as ds
import resnet_seg as res
import utils as utl

/home/qiang/anaconda3/envs/sfc/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qiang/anaconda3/envs/sfc/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qiang/anaconda3/envs/sfc/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qiang/anaconda3/envs/sfc/lib/py

In [2]:
import pyvista as pv
import numpy as np
from hilbertcurve.hilbertcurve import HilbertCurve

In [49]:
def batched_index_select(inputs, index):
    """

    :param inputs: torch.Size([batch_size, num_dims, num_vertices, 1])
    :param index: torch.Size([batch_size, num_vertices, k])
    :return: torch.Size([batch_size, num_dims, num_vertices, k])
    """
    batch_size, num_dims, num_vertices, _ = inputs.shape
    k = index.shape[2]
    idx = torch.arange(0, batch_size) * num_vertices
    idx = idx.view(batch_size, -1)

    inputs = inputs.transpose(2, 1).contiguous().view(-1, num_dims)
    index = index.contiguous().view(batch_size, -1) + idx.type(index.dtype).to(inputs.device)
    index = index.view(-1)

    return torch.index_select(inputs, 0, index).contiguous().view(batch_size, -1, num_dims).transpose(2, 1).contiguous().view(batch_size, num_dims, -1, k)

In [157]:
pos = torch.randn(4, 9, n_points)
print(pos)

batch, n_dim, n_points = pos.shape

tensor([[[ 1.9871, -0.3154, -1.1726,  ..., -1.6195, -0.8518,  1.0082],
         [-1.0009,  0.0994, -1.8702,  ..., -0.4132,  1.2127, -0.5906],
         [-0.5649, -0.9054, -0.3611,  ..., -1.7990, -0.2402, -2.2378],
         ...,
         [ 1.2326,  0.8871,  0.2282,  ...,  1.1615,  0.7533,  1.3328],
         [-1.0621, -0.8828, -0.8165,  ...,  0.6150,  0.2614, -0.5948],
         [ 0.9422, -0.6477,  0.1608,  ..., -0.0698,  0.0485,  0.4686]],

        [[ 0.4363,  1.3544,  0.5931,  ...,  0.8893,  0.3952,  0.3595],
         [-1.0674, -0.3457,  1.0209,  ...,  1.2282,  1.1285, -0.9224],
         [-0.7605,  1.5244, -0.0429,  ...,  1.6734,  0.6424, -0.4307],
         ...,
         [ 1.6779, -1.3695, -0.0711,  ...,  1.1475,  0.2194, -0.0367],
         [ 0.6787, -0.7417,  0.3600,  ..., -0.1595, -0.6209,  1.1902],
         [ 0.3752, -0.1914, -0.2131,  ...,  0.0760,  1.4133, -2.1128]],

        [[-0.5324, -0.4942,  0.6187,  ..., -0.7539,  1.0893,  1.3447],
         [ 1.6806,  0.1354, -1.6177,  ...,  0

In [158]:
def cal_conv_idx(n_points=1048, k=5):
    idx = torch.ones(n_points, k, dtype=torch.long)
    padding = k //2
    idx[:, 0] = torch.arange(-padding, n_points-padding, 1).type(torch.long)
    for i in range(1, k):
        idx[:, i] = idx[:, i-1]+1
    idx[idx>=n_points] = 2*(n_points-1) - idx[idx>=n_points]
    idx = torch.abs(idx)
    return idx 

In [159]:
idx = conv_idx(n_points, 5).unsqueeze(0).repeat(batch, 1, 1)
print(idx.shape, idx)

torch.Size([4, 1048, 5]) tensor([[[   2,    1,    0,    1,    2],
         [   1,    0,    1,    2,    3],
         [   0,    1,    2,    3,    4],
         ...,
         [1043, 1044, 1045, 1046, 1047],
         [1044, 1045, 1046, 1047, 1046],
         [1045, 1046, 1047, 1046, 1045]],

        [[   2,    1,    0,    1,    2],
         [   1,    0,    1,    2,    3],
         [   0,    1,    2,    3,    4],
         ...,
         [1043, 1044, 1045, 1046, 1047],
         [1044, 1045, 1046, 1047, 1046],
         [1045, 1046, 1047, 1046, 1045]],

        [[   2,    1,    0,    1,    2],
         [   1,    0,    1,    2,    3],
         [   0,    1,    2,    3,    4],
         ...,
         [1043, 1044, 1045, 1046, 1047],
         [1044, 1045, 1046, 1047, 1046],
         [1045, 1046, 1047, 1046, 1045]],

        [[   2,    1,    0,    1,    2],
         [   1,    0,    1,    2,    3],
         [   0,    1,    2,    3,    4],
         ...,
         [1043, 1044, 1045, 1046, 1047],
         [1

In [160]:
def cal_dis_corr(pos, idx):
    assert len(pos.shape) >= 3, 'length of input pos should be at least 3, [B, C, N]'
    if len(pos.shape)<4:
        pos = pos.unsqueeze(-1)
    batch = pos.shape[0]
    new_pos = batched_index_select(pos, idx)
#     print(new_pos)
    pos_diff = new_pos - new_pos[:, :, :, 2:3].repeat(1, 1, 1, 5)
    pos_diff = torch.sum(pos_diff ** 2, dim=1, keepdims=True)
    
#     print(pos_diff.shape, '\n', pos_diff)
    new_pos_dis = 1 - torch.div(pos_diff, torch.sum(pos_diff, dim =-1, keepdims=True))
    pos_dis_corr = torch.softmax(new_pos_dis, dim=-1)
    return pos_dis_corr.view(batch, 1, -1, 1)

In [161]:
dis_corr = cal_dis_corr(pos, idx)

In [162]:
print(dis_corr.shape)
print(dis_corr)

torch.Size([4, 1, 5240, 1])
tensor([[[[0.1751],
          [0.2037],
          [0.2425],
          ...,
          [0.2430],
          [0.1850],
          [0.1935]]],


        [[[0.2007],
          [0.1780],
          [0.2427],
          ...,
          [0.2429],
          [0.1840],
          [0.1945]]],


        [[[0.1798],
          [0.1989],
          [0.2428],
          ...,
          [0.2422],
          [0.1716],
          [0.2074]]],


        [[[0.2115],
          [0.1676],
          [0.2418],
          ...,
          [0.2430],
          [0.1926],
          [0.1859]]]])


In [163]:
pos.shape

torch.Size([4, 9, 1048])

In [165]:
torch.mul(pos, dis_corr.squeeze())

RuntimeError: The size of tensor a (1048) must match the size of tensor b (5240) at non-singleton dimension 2

In [147]:
pos.shape

torch.Size([2, 3, 1048])

In [149]:
new_pos = batched_index_select(pos.unsqueeze(-1), idx).view(-1, 3, n_points, )
print(new_pos.shape)

new_pos = torch.mul(new_pos, dis_corr)
print(new_pos.shape)

torch.Size([2, 3, 1048, 5])


RuntimeError: The size of tensor a (5) must match the size of tensor b (5240) at non-singleton dimension 3

In [2]:
root_dir = '/data/sfc/S3DIS'
batch_size = 32
in_channels = 9
kernel_size = 15

In [3]:
phases = ['train', 'test']
datasets, dataloaders, num_classes = ds.get_s3dis_dataloaders(root_dir=root_dir,
                                                              phases=phases,
                                                              batch_size=batch_size,
                                                              category=5)

In [4]:
device = torch.device('cuda:0')
model = res.resnet18(in_channels=in_channels, num_classes=num_classes,
                         kernel_size=kernel_size).to(device)

In [15]:
dataloader_iter = iter(dataloaders['train'])
data = next(dataloader_iter)

In [16]:
pos = data.pos
print('pos shape is ', pos.shape)
# dense points
points = torch.cat(((pos+1)/2, data.x[:, :, :3]), dim=2)[0]
points = points.numpy()
print('points dtype', points.shape, points.dtype)

pos shape is  torch.Size([32, 4096, 3])
points dtype (4096, 6) float32


In [19]:
point_cloud = pv.PolyData(points[:, :3])
point_cloud['color'] = points[:, 3:6]
point_cloud

PolyData (0x7fc949fab590)
  N Cells:	4096
  N Points:	4096
  X Bounds:	3.145e-01, 6.933e-01
  Y Bounds:	3.942e-01, 7.658e-01
  Z Bounds:	5.364e-07, 9.523e-01
  N Arrays:	1

In [20]:
voxel_size = 0.02
points_quantized = np.floor(points[:, :3] / voxel_size).astype(int)
points_quantized, quantization_indices = np.unique(points_quantized, return_index=True, axis=0)
points_voxelized = points[quantization_indices, :]
points_voxelized.shape
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
# dense points
point_cloud = pv.PolyData(points[:, :3])
point_cloud['color'] = points[:, 3:6]
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=10., render_points_as_spheres=True)
# voxelized points
point_cloud = pv.PolyData(points_voxelized[:, :3])
point_cloud['color'] = points_voxelized[:, 3:6]
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=30., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()

In [21]:
p = np.ceil(np.log2(points_quantized.max())).astype(int)
print('p is ', p)
hilbert_curve = HilbertCurve(p, 3)
hilbert_dist = np.zeros(points_quantized.shape[0])
for i in range(points_quantized.shape[0]):
    hilbert_dist[i] = hilbert_curve.distance_from_coordinates(points_quantized[i, :])
hilbert_idx = np.argsort(hilbert_dist)
points_voxelized = points_voxelized[hilbert_idx, :]
points_quantized = points_quantized[hilbert_idx, :]
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
# voxelized points
point_cloud = pv.PolyData(points_voxelized[:, :3])
point_cloud['color'] = points_voxelized[:, 3:6]
cells = np.full((points_voxelized.shape[0] - 1, 3), 2, dtype=np.int)
cells[:, 1] = np.arange(0, points_voxelized.shape[0] - 1, dtype=np.int)
cells[:, 2] = np.arange(1, points_voxelized.shape[0], dtype=np.int)
point_cloud.lines = cells
plotter.add_mesh(point_cloud, scalars='color', rgb=True, point_size=30., 
                 line_width=5., render_points_as_spheres=True)
plotter.show_grid()
plotter.show()


p is  6


In [14]:
def inside_cube(point, mins, maxs):
    isin = True
    for i in range(len(point)):
        if point[i] < mins[i] or point[i] > maxs[i]:
            isin = False
    return isin
kernel_points = np.mgrid[0:3, 0:3, 0:3].reshape((3, -1)).T
hilbert_dist = np.zeros(kernel_points.shape[0])
for i in range(kernel_points.shape[0]):
    hilbert_dist[i] = hilbert_curve.distance_from_coordinates(kernel_points[i, :])
idx = np.argsort(hilbert_dist)
kernel_points = kernel_points[idx, :]
center_point = points_quantized[100000, :]
print(center_point)
kernel_local = kernel_points + center_point - 1
kernel_min = kernel_local.min(axis=0)
kernel_max = kernel_local.max(axis=0)
points_in = np.array([points_quantized[i, :] for i in range(points_quantized.shape[0]) 
                      if inside_cube(points_quantized[i, :], kernel_min, kernel_max)])
points_in
plotter = pv.BackgroundPlotter()
plotter.enable_eye_dome_lighting()
point_cloud = pv.PolyData(points_quantized)
# point_cloud['color'] = np.arange(points_quantized.shape[0])
# cells = np.full((points_quantized.shape[0] - 1, 3), 2, dtype=np.int)
# cells[:, 1] = np.arange(0, points_quantized.shape[0] - 1, dtype=np.int)
# cells[:, 2] = np.arange(1, points_quantized.shape[0], dtype=np.int)
# point_cloud.lines = cells
plotter.add_mesh(point_cloud, color='maroon', point_size=10., line_width=5, render_points_as_spheres=True)
# plotter.add_mesh(point_cloud, scalar='color', point_size=10., line_width=5, render_points_as_spheres=True)
point_cloud = pv.PolyData(center_point)
plotter.add_mesh(point_cloud, color='green', point_size=30., render_points_as_spheres=True)
point_cloud = pv.PolyData(points_in)
cells = np.full((points_in.shape[0] - 1, 3), 2, dtype=np.int)
cells[:, 1] = np.arange(0, points_in.shape[0] - 1, dtype=np.int)
cells[:, 2] = np.arange(1, points_in.shape[0], dtype=np.int)
point_cloud.lines = cells
plotter.add_mesh(point_cloud, color='green', point_size=20., line_width=5.0, render_points_as_spheres=True)
box = pv.Box(bounds=(kernel_min[0], kernel_max[0], kernel_min[1], kernel_max[1], kernel_min[2], kernel_max[2]))
plotter.add_mesh(box, color='blue', show_edges=True, opacity=0.85)
plotter.show_grid()
plotter.show()

IndexError: index 100000 is out of bounds for axis 0 with size 779